## Demostration of a simple cosine similarity content-based recommendation system

In [9]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils.process_data import user_item_interaction_scores
from parquet_data_reader import ParquetDataReader
from models.content_based import CosineSimilarityContentBased

import polars as pl
import numpy as np
parquet_reader = ParquetDataReader()

### Reading data and preprocessing it

In [ ]:
train_behavior_df = parquet_reader.read_data("../../data/train/behaviors.parquet")
embeddings_df = parquet_reader.read_data("../../data/document_vector.parquet")
processed_behavior_df = user_item_interaction_scores(train_behavior_df)
processed_behavior_df.describe()

statistic,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",232887.0,70421.0,"""232887""",232887.0,69098.0,232887.0,232887.0,232887.0,232887.0,232887.0,16219.0,4673.0,6341.0,232887.0,232887.0,226669.0,206617.0
"""null_count""",0.0,162466.0,"""0""",0.0,163789.0,0.0,0.0,0.0,0.0,0.0,216668.0,228214.0,226546.0,0.0,0.0,6218.0,26270.0
"""mean""",2.8828e8,9.7654e6,"""2023-05-21 16:43:32.126653""",43.901806,99.566208,1.685521,null,null,1.3135e6,0.101337,0.10759,1.438048,55.013405,0.066212,989106.069712,68.649857,69.112617
"""std""",1.6832e8,129682.727142,null,90.299965,4.944662,0.519545,null,null,737020.473097,null,0.309871,1.436164,13.891898,null,568255.962255,172.63356,32.25351
"""min""",149474.0,3.198681e6,"""2023-05-18 07:00:01""",0.0,7.0,1.0,null,null,10068.0,0.0,0.0,0.0,0.0,0.0,406.0,0.0,0.0
"""25%""",1.42038746e8,9.772032e6,"""2023-05-19 18:51:21""",11.0,100.0,1.0,null,null,686661.0,null,0.0,0.0,50.0,null,500429.0,7.0,36.0
"""50%""",2.90333572e8,9.774297e6,"""2023-05-21 16:46:53""",21.0,100.0,2.0,null,null,1.328699e6,null,0.0,1.0,60.0,null,991121.0,21.0,80.0
"""75%""",4.33166835e8,9.776882e6,"""2023-05-23 10:46:53""",45.0,100.0,2.0,null,null,1.954176e6,null,0.0,3.0,60.0,null,1.477665e6,62.0,100.0
"""max""",5.80100697e8,9.780271e6,"""2023-05-25 06:59:58""",1799.0,100.0,3.0,null,null,2.590637e6,1.0,1.0,4.0,90.0,1.0,2.030118e6,1799.0,100.0


In [ ]:
content_based = CosineSimilarityContentBased(behavior_data=processed_behavior_df, item_data=embeddings_df)

recommendations = content_based.recommend(14241, 10)
score = content_based.score(14241, 6044108)

print("Recommendations for user 14241")
print(recommendations)
print("Score for user 14241 and item 6044108")
print(score)

/home/pedropca/Documents/Datatek/Recommender systems/TDT4215/recommender_system/models/content_based.py:78: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(user_vector, item_vector) / (np.linalg.norm(user_vector) * np.linalg.norm(item_vector))


[7160857,
 7789111,
 6044108,
 4925298,
 7124272,
 7592637,
 5574958,
 5622577,
 7141651,
 4340954]

0.9761224341910409